In [89]:
import pickle
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.decomposition import TruncatedSVD
from sklearn.decomposition import NMF

Load Cleaned Data

In [10]:
hikes_df = pickle.load(open('../src/cleaned_hike_desc.pickle', 'rb'))
hikes_df.set_index('hike_id', inplace=True)

reviews = pickle.load(open('../src/cleaned_reviews.pickle', 'rb'))

In [11]:
reviews.set_index(['hike_id', 'user_id'], inplace=True)
reviews.head()

user_desc  \
hike_id user_id                                                               
hike_1  steph-scott-2     Beautiful in fall. Went on a Tuesday late morn...   
        diana-pinho-3     went on a Tuesday AM around 9ish and didn't ex...   
        jason-pennacchio  Hiked Oct 2020, hiked from parking lot #3. Arr...   
        lolita-dickson    We started with the blue trail and finished wi...   
        allie-jay-1       Definitely start with the red trail , it’s ste...   

                                                            cleaned_reviews  
hike_id user_id                                                              
hike_1  steph-scott-2     beautiful fall go tuesday late morning midfall...  
        diana-pinho-3     go tuesday around ish do not expect see many p...  
        jason-pennacchio  hiked oct hike parking lot arrive lot full ext...  
        lolita-dickson    start blue trail finish red one slippery today...  
        allie-jay-1       definitely start red trail steep difficult get...

Set up Document Term Matrix with CountVectorizer

## SVD

In [60]:
def display_topics(model, feature_names, no_top_words, topic_names=None):
    for ix, topic in enumerate(model.components_):
        if not topic_names or not topic_names[ix]:
            print("\nTopic ", ix)
        else:
            print("\nTopic: '",topic_names[ix],"'")
        print(", ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]))

Trail Descriptions

In [80]:
vectorizer = TfidfVectorizer(max_features=2000,  min_df=10)
hikes_dtm = vectorizer.fit_transform(hikes_df['cleaned_descriptions'])

lsa = TruncatedSVD(5)
hikes_topic = lsa.fit_transform(hikes_dtm)

display_topics(lsa, vectorizer.get_feature_names(), 10)


Topic  0
trail, use, loop, also, able, dog, mile, near, locate, traffic

Topic  1
trip, nature, primarily, hiking, walk, bird, watch, use, run, set

Topic  2
rate, moderate, well, october, wild, flower, beautiful, use, features, difficult

Topic  3
chance, see, wildlife, features, lake, offer, rate, moderate, run, loop

Topic  4
wildlife, see, chance, offer, wild, flower, beautiful, bird, watch, good


*The terms "trail" and "also" shows up in multiple topics.  Maybe consider adding to the stop words.*
### Topic breakdown:
* Topic 0: Things the Trail offers
* Topic 1: Not much?
* Topic 2/3: Describe the trail features

In [68]:
# Descriptions
nmf_model = NMF(20)
trail_desc = nmf_model.fit_transform(hikes_dtm)
display_topics(nmf_model, vectorizer.get_feature_names(), 10)


Topic  0
trail, locate, near, mile, virginia, running, pennsylvania, appalachian, west, valley

Topic  1
trail, offer, number, option, activity, traffic, mile, locate, near, lightly

Topic  2
good, skill, level, feature, near, locate, mile, walk, trail, carolina

Topic  3
loop, trail, feature, locate, near, mile, waterfall, massachusetts, run, connecticut

Topic  4
leash, kept, must, dog, also, able, use, trail, virginia, pennsylvania

Topic  5
april, use, well, september, traffic, october, november, bird, heavily, watch

Topic  6
wild, flower, beautiful, feature, near, locate, mile, colorado, mountain, virginia

Topic  7
offer, wildlife, see, chance, near, locate, mile, lightly, back, virginia

Topic  8
forest, great, set, feature, near, locate, mile, virginia, massachusetts, walk

Topic  9
nature, trip, use, primarily, hiking, walk, bird, traffic, watch, mile

Topic  10
accessible, yearround, heavily, traffic, carolina, tennessee, feature, north, near, locate

Topic  11
rate, modera

Trail Reviews

In [81]:
vectorizer = TfidfVectorizer(max_features=2000, min_df=10, max_df=0.4)
reviews_dtm = vectorizer.fit_transform(reviews['cleaned_reviews'])

In [88]:
lsa = TruncatedSVD(5)
reviews_topics = lsa.fit_transform(reviews_dtm)
display_topics(lsa, vectorizer.get_feature_names(), 10)


Topic  0
great, view, not, beautiful, nice, easy, get, go, lot, good

Topic  1
great, view, beautiful, kid, easy, family, awesome, short, amazing, workout

Topic  2
nice, beautiful, easy, view, walk, short, waterfall, fall, scenery, quick

Topic  3
beautiful, view, worth, amazing, fall, love, absolutely, top, waterfall, lake

Topic  4
view, top, amazing, awesome, worth, good, steep, nice, climb, summit


Add hikes and trail to stopwords

In [87]:
# Descriptions
nmf_model = NMF(5)
trail_desc = nmf_model.fit_transform(reviews_dtm)

In [86]:
display_topics(nmf_model, vectorizer.get_feature_names(), 10)


Topic  0
not, go, get, lot, pron, mile, back, way, take, see

Topic  1
great, kid, view, dog, family, little, day, place, love, workout

Topic  2
nice, walk, little, short, really, wood, view, dog, place, loop

Topic  3
beautiful, fall, waterfall, love, scenery, lake, day, absolutely, view, place

Topic  4
view, top, amazing, awesome, worth, good, steep, short, climb, summit

Topic  5
easy, walk, pretty, kid, fun, follow, short, dog, well, family


* Topic 0: Not much
* Topic 1 & 2: Family Friendly
* Topic 3: Categorizes easier hikes
* Topic 4: Harder hikes with good payout

In [90]:
# LDA 
vectorizer = CountVectorizer(max_features=2000,  min_df=10)
hikes_dtm = vectorizer.fit_transform(hikes_df['cleaned_descriptions'])

lda_hikes = LatentDirichletAllocation(n_components=2)
lda_hikes.fit(hikes_dtm)

LatentDirichletAllocation(n_components=2)

In [92]:
import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()

pyLDAvis.sklearn.prepare(lda_hikes, hikes_dtm, vectorizer)

/Users/andrew/.vscode/extensions/ms-python.python-2020.10.332292344/pythonFiles/lib/python/past/types/oldstr.py:23: DeprecationWarning: invalid escape sequence \d
  """


PreparedData(topic_coordinates=              x    y  topics  cluster       Freq
topic                                           
0      0.056697  0.0       1        1  58.543866
1     -0.056697  0.0       2        1  41.456134, topic_info=           Term         Freq         Total Category  logprob  loglift
733       offer  7470.000000   7470.000000  Default  30.0000  30.0000
728      number  5394.000000   5394.000000  Default  29.0000  29.0000
741      option  5393.000000   5393.000000  Default  28.0000  28.0000
4      activity  5387.000000   5387.000000  Default  27.0000  27.0000
807   primarily  8724.000000   8724.000000  Default  26.0000  26.0000
...         ...          ...           ...      ...      ...      ...
409        good  2622.995906   5982.950063   Topic2  -4.2570   0.0559
1057        use  7549.946749  24679.023736   Topic2  -3.1997  -0.3039
833        rate  3077.471500   7942.509409   Topic2  -4.0972  -0.0676
667    moderate  2598.029415   6416.889144   Topic2  -4.2665  -0.0236
38         back  2001.023853   5450.513929   Topic2  -4.5276  -0.1215

[156 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
0         1  0.512601        able
0         2  0.487465        able
3         1  0.424634  accessible
3         2  0.575415  accessible
4         2  0.999991    activity
...     ...       ...         ...
1137      2  0.052958  woodbridge
1144      1  0.947014    wyanokie
1144      2  0.059188    wyanokie
1146      1  0.424124   yearround
1146      2  0.575919   yearround

[203 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 2])